### Ukrainian Annotation

In this notebook, we will try jp-errant on Ukrainian language dataset

In [1]:
import sys
sys.path.insert(0, '../')

In [2]:
import jp_errant

In [3]:
jp_errant.__version__

'3.0.1'

In [4]:
annotator = jp_errant.load(lang="uk") # Ukrainian

2025-02-13 20:14:33 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-02-13 20:14:33 INFO: Downloaded file to C:\Users\Minh UBC\stanza_resources\resources.json
2025-02-13 20:14:34 INFO: Loading these models for language: uk (Ukrainian):
| Processor | Package     |
---------------------------
| tokenize  | iu          |
| mwt       | iu          |
| pos       | iu_charlm   |
| lemma     | iu_nocharlm |
| depparse  | iu_charlm   |

2025-02-13 20:14:34 INFO: Using device: cuda
2025-02-13 20:14:34 INFO: Loading: tokenize
c:\Users\Minh UBC\miniconda3\Lib\site-packages\stanza\models\tokenization\trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functi

In [5]:
original  = 'Але у нашому спілкуванні , і нашим відвідуванням та бедсідуванням з Мей Марта , ні Гудлі Бенкс ні Я можемо дізнатись що одного з нас вона переважає .'
corrected = 'Але у нашому спілкуванні і наших відвідувань та бесідувань із Мей Марта , ні Гудлі Бенкс ні я не можемо дізнатись , що одного з нас вона переважає .'

In [6]:
# do tokenization
original_tokens = annotator.parse(original)
corrected_tokens = annotator.parse(corrected)

In [7]:
for item in original_tokens.iter_words():
    print(f"Text: {item.text} | Lemma: {item.lemma} | POS: {item.upos}")

Text: Але | Lemma: але | POS: CCONJ
Text: у | Lemma: у | POS: ADP
Text: нашому | Lemma: наш | POS: DET
Text: спілкуванні | Lemma: спілкування | POS: NOUN
Text: , | Lemma: , | POS: PUNCT
Text: і | Lemma: і | POS: CCONJ
Text: нашим | Lemma: наш | POS: DET
Text: відвідуванням | Lemma: відвідування | POS: NOUN
Text: та | Lemma: та | POS: CCONJ
Text: бедсідуванням | Lemma: бедсідування | POS: NOUN
Text: з | Lemma: з | POS: ADP
Text: Мей | Lemma: Мей | POS: PROPN
Text: Марта | Lemma: Март | POS: PROPN
Text: , | Lemma: , | POS: PUNCT
Text: ні | Lemma: ні | POS: CCONJ
Text: Гудлі | Lemma: Гудлі | POS: PROPN
Text: Бенкс | Lemma: Бенкс | POS: PROPN
Text: ні | Lemma: ні | POS: CCONJ
Text: Я | Lemma: я | POS: PRON
Text: можемо | Lemma: могти | POS: VERB
Text: дізнатись | Lemma: дізнатися | POS: VERB
Text: що | Lemma: що | POS: SCONJ
Text: одного | Lemma: один | POS: DET
Text: з | Lemma: з | POS: ADP
Text: нас | Lemma: ми | POS: PRON
Text: вона | Lemma: вона | POS: PRON
Text: переважає | Lemma: пер

In [8]:
# annotate 
edits = annotator.annotate(orig=original_tokens, 
                           cor=corrected_tokens
                           )

In [9]:
for edit_item in edits:
    print(edit_item)

Orig: [4, 5, ','], Cor: [4, 4, ''], Type: 'U:PUNCT'
Orig: [6, 7, 'нашим'], Cor: [5, 6, 'наших'], Type: 'R:DET -> DET'
Orig: [7, 8, 'відвідуванням'], Cor: [6, 7, 'відвідувань'], Type: 'R:NOUN -> NOUN'
Orig: [9, 10, 'бедсідуванням'], Cor: [8, 9, 'бесідувань'], Type: 'R:NOUN -> NOUN'
Orig: [10, 11, 'з'], Cor: [9, 10, 'із'], Type: 'R:ADP -> ADP'
Orig: [18, 19, 'Я'], Cor: [17, 18, 'я'], Type: 'R:PRON -> PRON'
Orig: [19, 19, ''], Cor: [18, 19, 'не'], Type: 'M:PART'
Orig: [21, 21, ''], Cor: [21, 22, ','], Type: 'M:PUNCT'


### Run command line parallel to m2

In [10]:
ORIGINAL_FILE = "../data/GEC_European_Datasets/Ukrainian/0108.src.txt"
CORRECTION_FILE = "../data/GEC_European_Datasets/Ukrainian/0108.a1.txt"
OUTPUT_FILE = "../data/GEC_European_Datasets/Ukrainian/0108-jp-errant.m2"
LANGUAGE = "uk"

In [11]:
!jp_errant_parallel --help

usage: jp_errant_parallel [-h] [options] -orig ORIG -cor COR [COR ...] [-tsv yes] -out OUT

Align parallel text files and extract and classify the edits.

options:
  -h, --help            show this help message and exit
  -orig ORIG            The path to the original text file.
  -cor COR [COR ...]    The paths to >= 1 corrected text files.
  -out OUT              The output filepath.
  -lang LANG            The 2-letter language code (default: en).
  -lev                  Align using standard Levenshtein (default: True).
  -merge {rules,all-split,all-merge,all-equal}
                        Choose a merging strategy for automatic alignment.
                        rules: Use a rule-based merging strategy (default)
                        all-split: Merge nothing: MSSDI -> M, S, S, D, I
                        all-merge: Merge adjacent non-matches: MSSDI -> M, SSDI
                        all-equal: Merge adjacent same-type non-matches: MSSDI -> M, SS, D, I


#### Process Ukrainian demo set

In [12]:
!jp_errant_parallel -orig {ORIGINAL_FILE} -cor {CORRECTION_FILE} -lang {LANGUAGE} -out {OUTPUT_FILE}

Loading resources...
Processing parallel files...


2025-02-13 20:14:46 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES

2025-02-13 20:14:46 INFO: Downloaded file to C:\Users\Minh UBC\stanza_resources\resources.json
2025-02-13 20:14:46 INFO: Loading these models for language: uk (Ukrainian):
| Processor | Package     |
---------------------------
| tokenize  | iu          |
| mwt       | iu          |
| pos       | iu_charlm   |
| lemma     | iu_nocharlm |
| depparse  | iu_charlm   |

2025-02-13 20:14:46 INFO: Using device: cuda
2025-02-13 20:14:46 INFO: Loading: tokenize
C:\Users\Minh UBC\miniconda3\Lib\site-packages\stanza\models\tokenization\trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during u